# Test single

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
import torch
from PIL import Image
from torchvision.transforms import ToTensor

from comer.datamodule import vocab
from comer.lit_comer import LitCoMER

ckpt = "lightning_logs/version_48/checkpoints/epoch=259-step=61880-val_ExpRate=0.4218.ckpt"
device = "cuda:0"

model = LitCoMER.load_from_checkpoint(ckpt, map_location=device)
model.eval()
model = model.to(device)

/bao/JapanLLM/Qwen3-Vl/LLaMA-Factory/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

image_path = "comer_data/data/val/img/val000001.bmp"

img = Image.open(image_path)
img_tensor = ToTensor()(img).to(device)
mask = torch.zeros_like(img_tensor, dtype=torch.bool)

with torch.no_grad():
    hyp = model.approximate_joint_search(img_tensor.unsqueeze(0), mask)[0]

pred_indices = hyp.seq
words = [
    vocab.idx2word[i]
    for i in pred_indices
    if i not in (vocab.PAD_IDX, vocab.SOS_IDX, vocab.EOS_IDX)
]

print("".join(words))

# Test Batch infer

In [148]:
# Load nhiều ảnh  
device = "cuda"
img_paths = ['comer_data/data/val/img/val121.bmp','comer_data/data/val/img/val111.bmp','comer_data/data/val/img/val116.bmp']  
imgs = []  
masks = []  
  
for img_path in img_paths:  
    img = Image.open(img_path)  
    img_tensor = ToTensor()(img)  
    imgs.append(img_tensor)  
  
# Tìm kích thước lớn nhất  
heights = [img.size(1) for img in imgs]  
widths = [img.size(2) for img in imgs]  
max_height = max(heights)  
max_width = max(widths)  
  
# Tạo batch với padding  
batch_size = len(imgs)  
batch_imgs = torch.zeros(batch_size, 1, max_height, max_width)  
batch_masks = torch.ones(batch_size, max_height, max_width, dtype=torch.bool)  
  
# Copy từng ảnh vào batch và cập nhật mask  
for idx, img in enumerate(imgs):  
    h, w = heights[idx], widths[idx]  
    batch_imgs[idx, :, :h, :w] = img  
    batch_masks[idx, :h, :w] = False  # False = vùng ảnh thực  
  
# Chuyển sang device và inference  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
batch_imgs = batch_imgs.to(device)  
batch_masks = batch_masks.to(device)  
  
hyps = model.approximate_joint_search(batch_imgs, batch_masks)
  
# Chuyển đổi kết quả  
for i, hyp in enumerate(hyps):  
    pred_latex = vocab.indices2label(hyp.seq)  
    print(f"Image {i+1}: {pred_latex}")

Image 1: 及 諸 官 門 令 人 守 之 百 官 進 朝 不 得 入 守 度 衛 符
Image 2: 人 心 中 異 之 帝 意 自 端 縣 不 擊 莫 氣 使 政
Image 3: 有 天 下 者 順 也 及 李 之 代 黎 陳 之 代 李 以 有
